In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [24]:
sentence = "Unsqueeze inserts a dimension of size 1 at the specified dim.".split()
print(sentence)

['Unsqueeze', 'inserts', 'a', 'dimension', 'of', 'size', '1', 'at', 'the', 'specified', 'dim.']


In [25]:
vocab = sorted(list(set(sentence))) #대문자가 소문자보다 코드번호가 작음
print(vocab)

['1', 'Unsqueeze', 'a', 'at', 'dim.', 'dimension', 'inserts', 'of', 'size', 'specified', 'the']


In [26]:
vocab_size=len(vocab)
vocab_size

11

In [27]:
word_to_index = {tkn:i for i, tkn in enumerate(vocab, 1)} #vocab : word의 집합을 만들어 놓은 것
print(word_to_index) #word가 key, index가 value
word_to_index['<unk>'] = 0
print(word_to_index)

{'1': 1, 'Unsqueeze': 2, 'a': 3, 'at': 4, 'dim.': 5, 'dimension': 6, 'inserts': 7, 'of': 8, 'size': 9, 'specified': 10, 'the': 11}
{'1': 1, 'Unsqueeze': 2, 'a': 3, 'at': 4, 'dim.': 5, 'dimension': 6, 'inserts': 7, 'of': 8, 'size': 9, 'specified': 10, 'the': 11, '<unk>': 0}


In [35]:
print(word_to_index['dim.'])
encode = [word_to_index[t] for t in sentence]
print(encode)

5
[2, 7, 3, 6, 8, 9, 1, 4, 11, 10, 5]


In [44]:
def build_data(sentence, word_to_index):
    encode = [word_to_index[t] for t in sentence]
    input_seq = encode[:-1]
    label_seq = encode[1:]
    input_seq = torch.LongTensor(input_seq).unsqueeze(0)
    label_seq =torch.LongTensor([label_seq])    #1차원을 2차원으로 만듦
    print(input_seq)
    print(label_seq)
    return input_seq, label_seq

In [45]:
X, Y = build_data(sentence, word_to_index)
print(X)
print(Y)

tensor([[ 2,  7,  3,  6,  8,  9,  1,  4, 11, 10]])
tensor([[ 7,  3,  6,  8,  9,  1,  4, 11, 10,  5]])
tensor([[ 2,  7,  3,  6,  8,  9,  1,  4, 11, 10]])
tensor([[ 7,  3,  6,  8,  9,  1,  4, 11, 10,  5]])


In [ ]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        
        #input_size(단어의 개수)만큼 embedding 층을 만들겠다.
        self.embedding_layer = nn.EmbeddingBag(num_embeddings=vocab_size, embedding_dim=input_size)
        
        self.rnn = nn.RNN(input_size, hidden_size, batch_first = batch_first)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x):
        output = self.embedding_layer(x)
        #embedding 층 : 크기 변화(배치 크기, 시퀀스 길이) => (배치 크기, 시권스 길이,임베딩 차원) - 이미지의 크기를 바꾸어 줌
        output, hidden = self.rnn(output)
        # RNN층 : 크기 변화(배치크기, 시퀀스 길이, 임베딩 차원) =>
        # output:(배치크기, 시퀀스 길이, 은닉층 크기)
        # hidden : (1, 배치크기, 은닉층 크기)
        
        output = self.fc(output)
        #크기 번화 : (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기 *시퀀스 길이, 단어장 크기)
        
        return output.view(-1, output.size(2))

In [46]:
vocab_size=len(word_to_index)
input_size = vocab_size
hidden_size=20

In [ ]:
model =Net(vocab_size, input_size, hidden_size, batch_first=True )
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [48]:
index_to_word ={v:k for k, v in word_to_index.items()}
print(index_to_word)

{1: '1', 2: 'Unsqueeze', 3: 'a', 4: 'at', 5: 'dim.', 6: 'dimension', 7: 'inserts', 8: 'of', 9: 'size', 10: 'specified', 11: 'the', 0: '<unk>'}


In [ ]:
decode = lambda y:[index_to_word.get(x) for x in y]
print(decode)

In [ ]:
for step in range(200):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_func(output, Y.view(-1))
    loss.backward()
    optimizer.step()
    
    pred = output.softmax(-1).argmax(-1).tolist()
    